In [2]:
import numpy as np 
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [4]:
rho = 0.85

In [12]:
N = np.random.normal(loc=0, scale=1, size=20)

In [13]:
cov = np.reshape(np.array([1 if i==j else rho for j in range(20) for i in range(20)]),(20,20))

In [14]:
N

array([ 1.99565484,  1.51594747,  0.38737231, -0.48411439, -0.45703445,
        1.57352611, -0.80284203, -1.01726507,  0.13479085,  1.46693169,
       -0.13861228,  1.15650808,  1.58128489, -0.70216114,  2.41164053,
        1.39274027, -0.36388827, -0.44699695, -0.05825088, -1.35378319])

In [15]:
cov

array([[1.  , 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85,
        0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85],
       [0.85, 1.  , 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85,
        0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85],
       [0.85, 0.85, 1.  , 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85,
        0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85],
       [0.85, 0.85, 0.85, 1.  , 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85,
        0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85],
       [0.85, 0.85, 0.85, 0.85, 1.  , 0.85, 0.85, 0.85, 0.85, 0.85, 0.85,
        0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85],
       [0.85, 0.85, 0.85, 0.85, 0.85, 1.  , 0.85, 0.85, 0.85, 0.85, 0.85,
        0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85],
       [0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 1.  , 0.85, 0.85, 0.85, 0.85,
        0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85, 0.85],
       [0.85, 0.85, 0.85, 0.85, 0.85, 0.8

In [17]:
def gaussian_copula(normals: float):
    return U

In [18]:
-50*np.log(gaussian_copula(N))

C:\Users\mahar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


array([-24.77033796, 182.58148973,          nan, -15.9911871 ,
                nan,  16.10175822, 104.73933971, -16.99065641,
        14.90367397,          nan,          nan, 203.12594138,
                nan, 171.23201468,          nan,          nan,
                nan,          nan, -37.18279833, 205.63989143])